In [2]:
# input from user
# regex = input("Enter your regex: ")

operators = {
    "+",
    "*",
    "(",
    ")",
    "|",
    ".",
    "[",
    "]",
    "?",
}

In [3]:
# a-z A-z 0-9
alpha_numerics = set(
    [chr(i) for i in range(48, 58)]
    + [chr(i) for i in range(65, 91)]
    + [chr(i) for i in range(97, 123)]
)

In [4]:
def is_valid_regex(in_regex):
    return (
        all([c in alpha_numerics or c in operators for c in in_regex])
        and in_regex.count("(") == in_regex.count(")")
        and in_regex.count("[") == in_regex.count("]")
    )

In [5]:
is_valid_regex("(regex][)")

True

In [21]:
# precedence from highest to lowest
# Closure (Kleene star) a*
# Concatenation ab
# Union a+b
# If the input symbol is a letter… append it directly to the output queue

# If the input symbol is an operator… if there exists an operator already on the top of the operator stack with higher
# or equal precedence than our current input symbol, remove the operator from the top of the operator stack and append it
# to the output queue.
# Do this until the current input symbol has a higher precedence than the symbol on the top of the operator stack,
# or the operator stack is empty.
# If the input symbol is an operator AND there is a left parenthesis on top of the stack… append the input symbol onto
# the stack on top of the left parenthesis.
# If the input symbol is an ( … append it to the operator stack
# If the input symbol is an ) … pop all operators from the operator stack and append them to the output queue
# until you find an ( . Then, you can remove both of those parentheses and continue with the algorithm.
operators_precedence = {
    "*": 5,
    "+": 4,
    "?": 3,
    ".": 2,
    "|": 1,
}


def regex_to_postfix(regex):
    stack = []
    output_queue = []

    for char in regex:
        if char in alpha_numerics:
            output_queue.append(char)
        elif char in operators:
            if stack:
                if (
                    stack[-1] in operators_precedence.keys()
                    and char in operators_precedence.keys()
                ):
                    # print("aho")
                    while (
                        stack
                        and stack[-1] in operators_precedence.keys()
                        and operators_precedence[stack[-1]]
                        >= operators_precedence[char]
                    ):

                        output_queue.append(stack.pop())

                    stack.append(char)

                elif char in operators_precedence.keys() and stack[-1] == "(":
                    stack.append(char)
                elif char == "(":
                    stack.append(char)
                elif char == ")":
                    while stack and stack[-1] != "(":
                        output_queue.append(stack.pop())
                    else:
                        if stack[-1] == "(":
                            stack.pop()
                        else:
                            return False

            else:
                stack.append(char)
    if "(" in stack or ")" in stack:
        return False
    while stack:
        top = stack[-1]
        if top == "(":
            return False
        output_queue.append(stack.pop())
    return output_queue


# regex = "(A+.B*)?.(C|D)"
regex = "(0.2)+++|C.D"

# if regex_to_postfix(regex) == False:
print(regex_to_postfix(regex))
# else:
#     print("tmam")

# regex
# operators_precedence.keys()

['0', '2', '.', '+', '+', '+', 'C', 'D', '.', '|']


In [7]:
# Algorithm
# ( -> push to stack
# Alphanumeric -> append to output queue
# operator
# {
#     "*" : 5,
#     "+" : 4,
#     "?" : 3,
#     "." : 2,
#     "|" : 1
# }
# [abc] -> a | b | c
# ) -> pop from stack until you find (
# ranges -

In [ ]:
"""
test cases
+++ -> invalid
[a]
"""